# Import

In [2]:
import numpy as np
import json

# Classes

In [3]:
# Convolution
class Convolution:
    def __init__(self, kernel, padding, stride, nb_filter, fct_activation):
        self.kernel = kernel
        self.padding = padding
        self.stride = stride
        self.nb_filter = nb_filter
        self.fct_activation = fct_activation

In [4]:
# input
class InputLayer(Convolution):
    def __init__(self, shape, kernel, padding, stride, nb_filter, fct_activation):
        self.shape = shape
        self.kernel = kernel
        self.padding = padding
        self.stride = stride
        self.nb_filter = nb_filter
        self.fct_activation = fct_activation

In [5]:
# Pooling Avg/Max
class Pooling:
    def __init__(self, op, kernel=2, padding="valid", stride=None):
        self.op = op # is the operation wanted (avg/max)
        self.kernel = kernel
        self.padding = padding
        self.stride = stride

In [6]:
# Class Flatten
class Flatten:
    pass

In [7]:
# Dense --> Fully connected layer
class Dense:
    def __init__(self, nb_neurones, fct_activation):
        self.nb_neurones = nb_neurones
        self.fct_activation = fct_activation

# Functions

In [17]:
# function that create CNN with list of layer
def create_CNN(list_layers):
    str_model_cnn = "model = keras.models.Sequential([\n"
    # iterate List of layer
    for layer in list_layers:
        
        # if is Intput
        if isinstance(layer, InputLayer):
            str_model_cnn += "\t\tkeras.layers.Conv2D({}, kernel_size={}, strides={},  activation={}, input_shape={}, padding={}),\n".format(
                    layer.nb_filter,
                    layer.kernel,
                    layer.stride,
                    layer.fct_activation,
                    layer.shape,
                    layer.padding)
            
            
        # if is Convolution
        elif isinstance(layer, Convolution):
            str_model_cnn += "\t\tkeras.layers.Conv2D({}, kernel_size={}, strides={}, activation={}, padding={}),\n".format(
                    layer.nb_filter,
                    layer.kernel,
                    layer.stride,
                    layer.fct_activation,
                    layer.padding)
            
        # if is Pooling
        elif isinstance(layer, Pooling):
            if(layer.op == "avg"): # avg Pooling 
                str_model_cnn += "\t\tkeras.layers.AveragePooling2D(pool_size={}, strides={}, padding={}),\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
                
            else : # Max Pooling
                str_model_cnn += "\t\tkeras.layers.MaxPooling2D(pool_size={}, strides={}, padding={}),\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
                
        # if is Dense (aka Fully connected layer)
        elif isinstance(layer, Dense):
            str_model_cnn += "\t\tkeras.layers.Dense({}, activation={}),\n".format(
                layer.nb_neurones, 
                layer.fct_activation)
            
        # if is flatten
        elif isinstance(layer, Flatten):
            str_model_cnn += "\t\tkeras.layers.Flatten(),\n"
            
        # Not possible
        else : print("Not Possible")

    # end model 
    str_model_cnn += "\n\t])\n"
    return str_model_cnn

In [18]:
# read json file 
# return list of layer
def getArchitectureFromJsonFile(json_file):
    
    # get architecture by json file
    with open(json_file) as archi_json:
        archi = json.load(archi_json)

    # instantiate class
    list_layer = []
    for layer in archi:
        parameters = layer[parameters] # get parameters
        # if is Input class
        if(layer[class] == "Input"):
            # verify if the padding is int or string
            try:
                padding = int(parameters[padding])
            except:
                padding = "" + parameters[padding] + ""
            
            # instantiate class and add into the list
            list_layer.append(InputLayer(parameters[shape], 
                                parameters[kernel], 
                                padding, 
                                parameters[stride],
                                parameters[nb_filter],
                                parameters[fct_activation]))

        
        # if is Pooling class
        elif(layer[class] == "Pooling"):
            try:
                padding = int(parameters[padding])
            except:
                padding = "" + parameters[padding] + ""
                
            list_layer.append(Pooling(parameters[op],
                                      parameters[kernel],
                                      padding,
                                      parameters[stride]))
            
        # if is Convolution class
        elif(layer["class"] == "Convolution"):
            try:
                padding = int(parameters[padding])
            except:
                padding = "" + parameters[padding] + ""
                
                
            list_layer.append(Convolution(parameters[kernel],
                                          padding,
                                          parameters[stride],
                                          parameters[nb_filter],
                                          parameters[fct_activation]))
            
        # if is Flatten class
        elif(layer["class"] == "Flatten"):
            list_layer.append(Flatten())
        
        # if is Dense class
        elif(layer["class"] == "Dense"):
            list_layer.append(Dense(parameters[nb_neurones],
                                    parameters[fct_activation]))

        else : print("Error")
    return list_layer

In [72]:
def create_py_file(json_file):
    s = json_file.split(".")
    file_name = s[0].split("/")[1]
    architecture = getArchitectureFromJsonFile(json_file)
    
    py_dir = "architecture_py/"
    # reset file
    file_py = open(py_dir + file_name + ".py", "w")
    file_py.close()
    
    file_py = open(py_dir + file_name + ".py", "a") # Open file in writting (a --> append)
    
    # write import
    file_py.write("""
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import plot_model
import sys
import traceback
""")
    
    # write train/test data 
    file_py.write("""(train_x, train_y), (test_x, test_y) = keras.datasets.mnist.load_data()

# normaliser les pixel 0-255 -> 0-1
train_x = train_x / 255.0
test_x = test_x / 255.0

train_x = tf.expand_dims(train_x, 3)
test_x = tf.expand_dims(test_x, 3)

val_x = train_x[:5000]
val_y = train_y[:5000]

""")
    
    # try
    file_py.write("""try:
    """)
    
    # write architecture model
    file_py.write(create_CNN(architecture))
    
    # write : create png of the model
    file_py.write("    plot_model(model, show_shapes=True, to_file=\"%s\")\n" % (file_name+".png"))
    
    # write compiler
    file_py .write("""    model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])\n""")
    
    # write model training
    file_py.write("""    model.fit(train_x, train_y, epochs=5, validation_data=(val_x, val_y))\n""")
    
    # write model evaluation
    file_py.write("""    print(model.evaluate(test_x, test_y))\n""")
    
    log_file = file_name +".log"
    file_py.write("""
    print('OK: file """ + log_file +""" has been create')
    log_file = open(\"""" + log_file + """\" , "w")
    log_file.write(str(model.evaluate(test_x, test_y)))
    log_file.close()
""")
    
    # 
    error_file = file_name+"_error.log"
    file_py.write("""except:
    print('error: file """ + error_file +""" has been create')
    error_file = open(\"""" + error_file + """\" , "w")
    traceback.print_exc(file=error_file)
    error_file.close()
""")
    
    # close
    file_py.close()

# Creation CNN valide

In [11]:
create_py_file("architecture_valid.json")

# Creation CNN non valide

In [12]:
create_py_file("architecture_invalid.json")

# Create CNN json

In [73]:
directory = "architecture_json/"
file_name = "archi_random_1.json"

In [74]:
create_py_file(directory+file_name)